In [72]:
from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn import neighbors

import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import seaborn as sns
%matplotlib inline

In [3]:
train = pickle.load(open('train_processed.pkl','rb'))
train.head()

,Dates,Category,Address,X,Y,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
0,2015-05-13 23:53:00,WARRANTS,OAK ST / LAGUNA ST,-122.425892,37.774599,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2015-05-13 23:53:00,OTHER OFFENSES,OAK ST / LAGUNA ST,-122.425892,37.774599,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2015-05-13 23:33:00,OTHER OFFENSES,VANNESS AV / GREENWICH ST,-122.424363,37.800414,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2015-05-13 23:30:00,LARCENY/THEFT,1500 Block of LOMBARD ST,-122.426995,37.800873,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2015-05-13 23:30:00,LARCENY/THEFT,100 Block of BRODERICK ST,-122.438738,37.771541,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [4]:
test = pickle.load(open('test_processed.pkl','rb'))
test.head()

,Dates,Address,X,Y,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
Id,,,,,,,,,,,,,,
0,2015-05-10 23:59:00,2000 Block of THOMAS AV,-122.399588,37.735051,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015-05-10 23:51:00,3RD ST / REVERE AV,-122.391523,37.732432,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-05-10 23:50:00,2000 Block of GOUGH ST,-122.426002,37.792212,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2015-05-10 23:45:00,4700 Block of MISSION ST,-122.437394,37.721412,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-05-10 23:45:00,4700 Block of MISSION ST,-122.437394,37.721412,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
# encode the category
trainset = pd.read_csv('train.csv', parse_dates = ['Dates'])
testset = pd.read_csv('test.csv', parse_dates = ['Dates'])
label = preprocessing.LabelEncoder()
crime = label.fit_transform(trainset.Category) 

train = pd.concat([pd.get_dummies(trainset.DayOfWeek), pd.get_dummies(trainset.PdDistrict), pd.get_dummies(trainset.Dates.dt.hour)], axis=1)    
train['crime'] = crime
test = pd.concat([pd.get_dummies(testset.DayOfWeek), pd.get_dummies(testset.PdDistrict), pd.get_dummies(testset.Dates.dt.hour)], axis=1)   


In [61]:
# split into training set and test set
training, validation = train_test_split(train, train_size=0.5)# TODO: gridsearchCV

feature_list = training.columns.tolist()   
feature_list = feature_list[:len(feature_list) - 1] 

## Bayesian Model

In [70]:
# Naïve Bayes Model
nb_model = GaussianNB()
nb_model.fit(training[feature_list], training['crime'])
print('Naïve Bayes accuracy for train set: %f' % nb_model.score(training[feature_list], training['crime']))

# test on the test set 
test_predicted = np.array(nb_model.predict_proba(test[feature_list])) 

Naïve Bayes accuracy for train set: 0.002082


In [65]:
nb_model = BernoulliNB()
nb_model.fit(training[feature_list], training['crime'])   
predicted = np.array(model.predict_proba(validation[feature_list]))   
print('Naïve Bayes accuracy for train set: %f' % nb_model.score(training[feature_list], training['crime']))

# test on the test set 
test_predicted = np.array(nb_model.predict_proba(test[feature_list])) 

Naïve Bayes accuracy for train set: 0.224423


## Logistic Regression Model

In [78]:
# TODO: test different C values
lr_model= LogisticRegression(C=0.5)
lr_model.fit(training[feature_list], training['crime'])
predicted = np.array(model_LR.predict_proba(validation[feature_list]))
print('Logistic Regression accuracy for train set: %f' % lr_model.score(training[feature_list], training['crime']))

# test on the test set 
test_predicted = np.array(lr_model.predict_proba(test[feature_list])) 

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Regression accuracy for train set: 0.224282


## Random Forest Model

In [75]:
rf_model = RandomForestClassifier()  
rf_model.fit(training[feature_list], training['crime'])
predicted = np.array(rf_model.predict_proba(validation[feature_list]))
print('Random Forest accuracy for train set: %f' % rf_model.score(training[feature_list], training['crime']))

# test on the test set 
test_predicted = np.array(rf_model.predict_proba(test[feature_list])) 

C:\Users\user\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random Forest accuracy for train set: 0.230580


## KNN model

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors = 10)
knn_model = knn.fit(training[feature_list], training['crime'])
print('NN accuracy for train set: %f' % knn_model.score(training[feature_list], training['crime']))